In [ ]:
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [ ]:
import torch
import torchvision
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms,utils
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split,Dataset
from tqdm.notebook import tqdm as tqdm
import torch.utils.data as data
import torchvision.models as models
import seaborn as sns
import torch.quantization as quantization
import warnings
import pickle
import cv2
warnings.filterwarnings('ignore')
sns.set_theme()
import torchvision.transforms as transforms
from torchvision import datasets, transforms
%matplotlib inline
plt.ion() #Turn interactive mode on.
torch.manual_seed(34)
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import multilabel_confusion_matrix
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import time
import tensorflow as tf

In [ ]:
# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
if not os.path.exists('./datasets/visdrone_new/'):
    os.mkdir('./datasets/visdrone_new/')

In [ ]:
train_root_dir = './datasets/visdrone_new/VisDrone2019-DET-train'
val_root_dir = './datasets/visdrone_new/VisDrone2019-DET-val'
test_root_dir = './datasets/visdrone_new/'

In [ ]:
image_list_train = sorted(os.listdir(os.path.join(train_root_dir, 'images')))
label_list_train = sorted(os.listdir(os.path.join(train_root_dir, 'annotations')))

image_list_val = sorted(os.listdir(os.path.join(val_root_dir, 'images')))
label_list_val = sorted(os.listdir(os.path.join(val_root_dir, 'annotations')))

image_list_test = sorted(os.listdir(os.path.join(test_root_dir, 'images')))
label_list_test = sorted(os.listdir(os.path.join(test_root_dir, 'annotations')))

In [ ]:
print(len(image_list_train))
print(len(image_list_val))
print(len(image_list_test))

6471
548
1610


In [ ]:
np.array(image_list_train).shape

(6471,)

In [ ]:
def image_label_pairs(image_list,label_list,root_dir):
  img_label_pair = {}
  for idx in range(len(image_list)):
    img_path = os.path.join(root_dir, 'images', image_list[idx])
    label_path = os.path.join(root_dir, 'annotations', label_list[idx])
    img = Image.open(img_path)
    img = img.convert('RGB')
    classes = {}
    with open(label_path, 'r') as f:
      for line in f:
        l2 = line.split(',')
        label = l2[-2]
        if l2[-2] not in classes:
          classes[l2[-2]] = 1
        else:
          classes[l2[-2]] += 1
    max_value = max(classes.values())
    max_keys = [k for k, v in classes.items() if v == max_value]
    label = max_keys[0]

    img_label_pair[idx] = []
    img_label_pair[idx].append(img)
    img_label_pair[idx].append(int(label))

  return img_label_pair

In [ ]:
img_label_pairs_train = image_label_pairs(image_list_train,label_list_train,train_root_dir)

In [ ]:
img_label_pairs_val = image_label_pairs(image_list_val,label_list_val,val_root_dir)

In [ ]:
img_label_pairs_test = image_label_pairs(image_list_test,label_list_test,test_root_dir)

In [ ]:
def reshape_img(image, target_shape):
    resized_image = cv2.resize(image, target_shape, interpolation=cv2.INTER_AREA)

    if resized_image.shape[0] < target_shape[0] or resized_image.shape[1] < target_shape[1]:
        pad_height = max(target_shape[0] - resized_image.shape[0], 0)
        pad_width = max(target_shape[1] - resized_image.shape[1], 0)
        resized_image = cv2.copyMakeBorder(
            resized_image, 0, pad_height, 0, pad_width,
            cv2.BORDER_CONSTANT, value=(0, 0, 0)
        )

    return resized_image

In [ ]:
def xdata_ydata_array(img_label_pairs):
  x_data = []
  y_data = []
  for idx in range(len(img_label_pairs)):
    image = np.array(img_label_pairs[idx][0])
    image = reshape_img(image, (224, 224))
    x_data.append(image)
    y_data.append(img_label_pairs[idx][1])

  x_data = np.array(x_data)
  y_data = np.array(y_data)
  y_data = tf.one_hot(y_data.astype(np.int32), depth=2)
  return x_data,y_data

In [ ]:
x_train,y_train = xdata_ydata_array(img_label_pairs_train)

2023-11-02 22:29:12.622453: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 22:29:13.619780: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21851 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:98:00.0, compute capability: 8.6


In [ ]:
x_val,y_val = xdata_ydata_array(img_label_pairs_val)

In [ ]:
x_test,y_test = xdata_ydata_array(img_label_pairs_test)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# Create a MobileNetV2 base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(2)

model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Save the model
model.save('./models/mobilenetv2_visdrone.h5')

Epoch 1/5


2023-10-30 16:56:16.143090: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8500
2023-10-30 16:56:17.784084: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


203/203 [==============================] - 34s 123ms/step - loss: 0.0185 - accuracy: 0.9975 - val_loss: 0.1806 - val_accuracy: 0.9944
Epoch 2/5
203/203 [==============================] - 24s 116ms/step - loss: 0.0146 - accuracy: 0.9988 - val_loss: 0.2173 - val_accuracy: 0.9944
Epoch 3/5
203/203 [==============================] - 24s 116ms/step - loss: 0.0171 - accuracy: 0.9988 - val_loss: 0.2847 - val_accuracy: 0.9944
Epoch 4/5
203/203 [==============================] - 24s 120ms/step - loss: 0.0106 - accuracy: 0.9988 - val_loss: 0.3509 - val_accuracy: 0.9944
Epoch 5/5
203/203 [==============================] - 24s 118ms/step - loss: 0.0149 - accuracy: 0.9988 - val_loss: 0.2453 - val_accuracy: 0.9944


In [ ]:
# Create a VGG_16 base model
base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(2)

model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Save the model
model.save('./models/vgg16_visdrone.h5')

Epoch 1/5


2023-10-30 18:53:03.484763: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8500
2023-10-30 18:53:04.791932: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


203/203 [==============================] - 36s 152ms/step - loss: 34706.1250 - accuracy: 0.9947 - val_loss: 0.1383 - val_accuracy: 0.9944
Epoch 2/5
203/203 [==============================] - 30s 146ms/step - loss: 0.2505 - accuracy: 0.9988 - val_loss: 0.2448 - val_accuracy: 0.9944
Epoch 3/5
203/203 [==============================] - 30s 148ms/step - loss: 0.0663 - accuracy: 0.9988 - val_loss: 0.1731 - val_accuracy: 0.9944
Epoch 4/5
203/203 [==============================] - 30s 147ms/step - loss: 0.0699 - accuracy: 0.9988 - val_loss: 0.1010 - val_accuracy: 0.9944
Epoch 5/5
203/203 [==============================] - 30s 148ms/step - loss: 0.0340 - accuracy: 0.9988 - val_loss: 0.1641 - val_accuracy: 0.9944


In [ ]:
import tensorflow as tf

def create_cnn_model(input_shape=(224, 224, 3), num_classes=2):
    model = tf.keras.Sequential()

    # Layer 1
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', input_shape=input_shape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(1, 1), strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(1, 1), strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())

    # Layer 2
    model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(128, kernel_size=(1, 1), strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(128, kernel_size=(1, 1), strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())

    # Layer 3
    model.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(256, kernel_size=(1, 1), strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(256, kernel_size=(1, 1), strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())

    # Layer 4
    model.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(512, kernel_size=(1, 1), strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(512, kernel_size=(1, 1), strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())

    # Layer 5
    model.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(512, kernel_size=(1, 1), strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(512, kernel_size=(1, 1), strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())

    # Flatten and fully connected layers
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(num_classes))

    return model

In [ ]:
# Define your CNN model
model = create_cnn_model()

model.summary()
model.save('./models/cnn.h5')

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_69 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization_75 (Bat  (None, 224, 224, 64)     256       
 chNormalization)                                                
                                                                 
 re_lu_83 (ReLU)             (None, 224, 224, 64)      0         
                                                                 
 conv2d_70 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 batch_normalization_76 (Bat  (None, 224, 224, 64)     256       
 chNormalization)                                                
                                                                 
 re_lu_84 (ReLU)             (None, 224, 224, 64)     

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 2

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_accuracy}")

In [ ]:
def AttentionBlock(in_channels):
    conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
    pool = nn.AdaptiveAvgPool2d(1)
    fc1 = nn.Linear(in_channels // 8, in_channels)
    fc2 = nn.Linear(in_channels, in_channels)
    relu = nn.ReLU()
    sigmoid = nn.Sigmoid()

    def forward(x):
        y = conv(x)
        y = relu(y)
        y = pool(y)
        y = y.view(y.size(0), -1)
        y = fc1(y)
        y = relu(y)
        y = fc2(y)
        y = sigmoid(y)
        return x * y.unsqueeze(2).unsqueeze(3)

    return forward

def AttentionCNN(flatten_shape=512*7*7, num_classes=2, in_channels=3):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', input_shape=(224, 224, in_channels)),
        AttentionBlock(64),
        tf.keras.layers.Conv2D(128, kernel_size=3, strides=2, padding='same'),
        AttentionBlock(128),
        tf.keras.layers.Conv2D(256, kernel_size=3, strides=2, padding='same'),
        AttentionBlock(256),
        tf.keras.layers.Conv2D(512, kernel_size=3, strides=2, padding='same'),
        AttentionBlock(512),
        tf.keras.layers.Conv2D(512, kernel_size=3, strides=2, padding='same'),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(4096),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dense(2048),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dense(num_classes)
    ])
    return model

# Create an instance of the AttentionCNN model
model = AttentionCNN()
model.summary()

TypeError: The added layer must be an instance of class Layer. Received: layer=<function AttentionBlock.<locals>.forward at 0x7f9555c7f820> of type <class 'function'>.

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

# Define the AttentionCNN model
model = AttentionCNN()

# Define the loss function and optimizer
loss_fn = SparseCategoricalCrossentropy(from_logits=True)
optimizer = Adam(learning_rate=0.001)

# Define metrics for evaluation
metrics = [SparseCategoricalAccuracy()]

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

# Training configuration
batch_size = 32
epochs = 10

# Training the model
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Print the test accuracy
print(f'Test accuracy: {test_accuracy}')


In [ ]:
class VisDroneDataset(Dataset):
    def __init__(self, root_dir_name, transform=None):
        self.transform = transform
        self.root_dir_name = root_dir_name
        if (root_dir_name == "train"):
            self.img_label_pairs = img_label_pairs_train
        elif (root_dir_name == "val"):
            self.img_label_pairs = img_label_pairs_val
        else:
            self.img_label_pairs = img_label_pairs_test

    def __len__(self):
        return len(self.img_label_pairs)

    def __getitem__(self, idx):
        img = self.img_label_pairs[idx][0]
        label = self.img_label_pairs[idx][1]

        if self.transform:
            img = self.transform(img)
        return img,label


train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

train_dataset = VisDroneDataset("train", train_transform)
val_dataset = VisDroneDataset("val", test_transform)
test_dataset = VisDroneDataset("test", test_transform)

In [ ]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class AttentionBlock(nn.Module):
    def __init__(self, in_channels):
        super(AttentionBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(in_channels // 8, in_channels)
        self.fc2 = nn.Linear(in_channels, in_channels)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.conv(x)
        y = self.relu(y)
        y = self.pool(y)
        y = y.view(y.size(0), -1)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y)
        return x * y.unsqueeze(2).unsqueeze(3)

In [ ]:
class AttentionCNN_v2(nn.Module):
    def __init__(self, flatten_shape=7*7*512, num_classes=2):
        super(AttentionCNN_v2, self).__init__()
        self.flatten_shape = flatten_shape
        self.conv1 = self.convLayer(3, 64, kernel_size=7, stride=2, padding=3)
        self.attention1 = AttentionBlock(64)
        self.conv2 = self.convLayer(64, 128, kernel_size=3, stride=2, padding=1)
        self.attention2 = AttentionBlock(128)
        self.conv3 = self.convLayer(128, 256, kernel_size=3, stride=2, padding=1)
        self.attention3 = AttentionBlock(256)
        self.conv4 = self.convLayer(256, 512, kernel_size=3, stride=2, padding=1)
        self.attention4 = AttentionBlock(512)
        self.conv5 = self.convLayer(512, 512, kernel_size=3, stride=2, padding=1)
        self.fc = nn.Linear(self.flatten_shape, num_classes)

    def convLayer(self,in_planes,out_planes,kernel_size,stride, padding, fl = 1):
        if fl:
            return nn.Sequential(
                nn.Conv2d(in_planes,in_planes,kernel_size=3, stride = 2, padding=1, groups=in_planes),
                nn.Conv2d(in_planes, out_planes, kernel_size=1),
                nn.BatchNorm2d(out_planes)
            )
        else:
            return nn.Sequential(
                nn.Conv2d(in_channels=in_planes, out_channels=out_planes, kernel_size=kernel_size, padding=padding,stride=stride),
                nn.BatchNorm2d(out_planes)
            )

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.attention1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.attention2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.attention3(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.attention4(x)
        x = self.conv5(x)
        x = F.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [ ]:
attention_model = AttentionCNN_v2()

In [ ]:
print_model_size(attention_model)

3.77 MB


In [ ]:
print_model_size(attention_model.to(dtype=torch.float64))

7.53 MB


In [ ]:
print_model_size(attention_model.to(dtype=torch.float16))

1.90 MB


In [ ]:
attention_model.train()

AttentionCNN_v2(
  (conv1): Sequential(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3)
    (1): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (attention1): AttentionBlock(
    (conv): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
    (pool): AdaptiveAvgPool2d(output_size=1)
    (fc1): Linear(in_features=8, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=64, bias=True)
    (relu): ReLU()
    (sigmoid): Sigmoid()
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64)
    (1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (attention2): AttentionBlock(
    (conv): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
    (pool): AdaptiveAvgPool2d(output_size=1)
    (fc1): Lin

In [ ]:
# Train the model
def train(epoch, model, trainloader, testloader, device, datatype,optimizer,criterion):
    for epoch in range(epoch):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader, 0):
            inputs, labels = inputs.to(dtype=datatype).to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.to(dtype=torch.float32)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 50 == 49:
                print('[Epoch %d, Batch %d] Loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0
        evaluate(model, testloader, device, datatype)

    return model

def evaluate(model, testloader, device, datatype):
    # Evaluate the model on the test set
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(dtype=datatype).to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.to(dtype=torch.float32).data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Testing Accuracy: %d %%' % (100 * correct / total))


### For 64 bits

In [ ]:
model = AttentionCNN_v2().to(dtype=torch.float64).to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
start_time = time.time()

train(10, model, train_loader, test_loader, device, torch.float64,optimizer,criterion)

end_time = time.time()
elapsed_time = end_time - start_time

print("Training time: {:.2f} seconds".format(elapsed_time))

[Epoch 1, Batch 50] Loss: 0.045
[Epoch 1, Batch 100] Loss: 0.056
[Epoch 1, Batch 150] Loss: 0.021
[Epoch 1, Batch 200] Loss: 0.005
Testing Accuracy: 99 %
[Epoch 2, Batch 50] Loss: 0.003
[Epoch 2, Batch 100] Loss: 0.009
[Epoch 2, Batch 150] Loss: 0.005
[Epoch 2, Batch 200] Loss: 0.006
Testing Accuracy: 99 %
[Epoch 3, Batch 50] Loss: 0.007
[Epoch 3, Batch 100] Loss: 0.003
[Epoch 3, Batch 150] Loss: 0.005
[Epoch 3, Batch 200] Loss: 0.005
Testing Accuracy: 99 %
[Epoch 4, Batch 50] Loss: 0.005
[Epoch 4, Batch 100] Loss: 0.005
[Epoch 4, Batch 150] Loss: 0.009
[Epoch 4, Batch 200] Loss: 0.001
Testing Accuracy: 99 %
[Epoch 5, Batch 50] Loss: 0.005
[Epoch 5, Batch 100] Loss: 0.000
[Epoch 5, Batch 150] Loss: 0.004
[Epoch 5, Batch 200] Loss: 0.089
Testing Accuracy: 99 %
[Epoch 6, Batch 50] Loss: 0.034
[Epoch 6, Batch 100] Loss: 0.032
[Epoch 6, Batch 150] Loss: 0.011
[Epoch 6, Batch 200] Loss: 0.029
Testing Accuracy: 99 %
[Epoch 7, Batch 50] Loss: 0.010
[Epoch 7, Batch 100] Loss: 0.006
[Epoch 7, B

In [ ]:
torch.save(model.state_dict(), "./models/attention_cnn_64bit.pth")

### For 32 bits

In [ ]:
model = AttentionCNN_v2().to(dtype=torch.float32).to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time

start_time = time.time()

train(10, model, train_loader, test_loader, device, torch.float32,optimizer,criterion)

end_time = time.time()
elapsed_time = end_time - start_time

print("Training time: {:.2f} seconds".format(elapsed_time))

[Epoch 1, Batch 50] Loss: 0.099
[Epoch 1, Batch 100] Loss: 0.000
[Epoch 1, Batch 150] Loss: 0.073
[Epoch 1, Batch 200] Loss: 0.015
Testing Accuracy: 99 %
[Epoch 2, Batch 50] Loss: 0.005
[Epoch 2, Batch 100] Loss: 0.031
[Epoch 2, Batch 150] Loss: 0.007
[Epoch 2, Batch 200] Loss: 0.003
Testing Accuracy: 99 %
[Epoch 3, Batch 50] Loss: 0.005
[Epoch 3, Batch 100] Loss: 0.001
[Epoch 3, Batch 150] Loss: 0.009
[Epoch 3, Batch 200] Loss: 0.006
Testing Accuracy: 99 %
[Epoch 4, Batch 50] Loss: 0.001
[Epoch 4, Batch 100] Loss: 0.012
[Epoch 4, Batch 150] Loss: 0.003
[Epoch 4, Batch 200] Loss: 0.004
Testing Accuracy: 99 %
[Epoch 5, Batch 50] Loss: 0.008
[Epoch 5, Batch 100] Loss: 0.014
[Epoch 5, Batch 150] Loss: 0.021
[Epoch 5, Batch 200] Loss: 0.001
Testing Accuracy: 99 %
[Epoch 6, Batch 50] Loss: 0.005
[Epoch 6, Batch 100] Loss: 0.018
[Epoch 6, Batch 150] Loss: 0.012
[Epoch 6, Batch 200] Loss: 0.003
Testing Accuracy: 99 %
[Epoch 7, Batch 50] Loss: 0.000
[Epoch 7, Batch 100] Loss: 0.000
[Epoch 7, B

In [ ]:
torch.save(model.state_dict(), "./models/attention_cnn_32bit.pth")

In [ ]:
from torchsummary import summary
# summary(cnn_model,(3,64,64))
model = AttentionCNN_v4(num_classes = 2).to(device)
summary(model,(3,64, 64))

NameError: name 'AttentionCNN_v4' is not defined

### For 16 bits

In [ ]:
model = AttentionCNN_v2().to(dtype=torch.float16).to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time

start_time = time.time()

train(10, model, train_loader, test_loader, device, torch.float16,optimizer,criterion)

end_time = time.time()
elapsed_time = end_time - start_time

print("Training time: {:.2f} seconds".format(elapsed_time))

[Epoch 1, Batch 50] Loss: nan
[Epoch 1, Batch 100] Loss: nan
[Epoch 1, Batch 150] Loss: nan
[Epoch 1, Batch 200] Loss: nan
Testing Accuracy: 99 %
[Epoch 2, Batch 50] Loss: nan
[Epoch 2, Batch 100] Loss: nan
[Epoch 2, Batch 150] Loss: nan
[Epoch 2, Batch 200] Loss: nan
Testing Accuracy: 99 %
[Epoch 3, Batch 50] Loss: nan
[Epoch 3, Batch 100] Loss: nan
[Epoch 3, Batch 150] Loss: nan
[Epoch 3, Batch 200] Loss: nan
Testing Accuracy: 99 %
[Epoch 4, Batch 50] Loss: nan
[Epoch 4, Batch 100] Loss: nan
[Epoch 4, Batch 150] Loss: nan
[Epoch 4, Batch 200] Loss: nan
Testing Accuracy: 99 %
[Epoch 5, Batch 50] Loss: nan
[Epoch 5, Batch 100] Loss: nan
[Epoch 5, Batch 150] Loss: nan
[Epoch 5, Batch 200] Loss: nan
Testing Accuracy: 99 %
[Epoch 6, Batch 50] Loss: nan
[Epoch 6, Batch 100] Loss: nan
[Epoch 6, Batch 150] Loss: nan
[Epoch 6, Batch 200] Loss: nan
Testing Accuracy: 99 %
[Epoch 7, Batch 50] Loss: nan
[Epoch 7, Batch 100] Loss: nan
[Epoch 7, Batch 150] Loss: nan
[Epoch 7, Batch 200] Loss: nan
T

In [ ]:
torch.save(model.state_dict(), "./models/attention_cnn_16bit.pth")

In [ ]:
# accuracies = [66, 72, 74]
# bits = [16, 32, 64]

# plt.plot(bits, accuracies)
# plt.xlabel("Bits")
# plt.ylabel("Accuracy")
# plt.show()

In [ ]:
# training_time = [374.56, 695.16, 927.15]
# bits = [16, 32, 64]

# plt.plot(bits, training_time)
# plt.xlabel("Bits")
# plt.ylabel("Training Time")
# plt.show()